<a href="https://colab.research.google.com/github/xjseabrum/lyrics-nlp-acoustic-predictions/blob/main/Models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import Statements and Setup


In [ ]:
!python --version

Python 3.7.13


In [ ]:
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [ ]:
import os as os
os.chdir("/content/gdrive/MyDrive/lyrics-nlp")

In [ ]:
!pip3 install -r requirements.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 68 kB 3.2 MB/s 
     |████████████████████████████████| 24.1 MB 17 kB/s 
     |████████████████████████████████| 59 kB 6.7 MB/s 
     |████████████████████████████████| 11.2 MB 27.4 MB/s 
     |████████████████████████████████| 1.4 MB 50.2 MB/s 
     |████████████████████████████████| 285 kB 51.9 MB/s 
     |████████████████████████████████| 256 kB 47.6 MB/s 
     |████████████████████████████████| 85 kB 5.1 MB/s 
     |████████████████████████████████| 4.7 MB 41.2 MB/s 
  Using cached pybind11-2.10.0-py3-none-any.whl (213 kB)
     |████████████████████████████████| 950 kB 45.9 MB/s 
     |████████████████████████████████| 62 kB 1.6 MB/s 
     |████████████████████████████████| 246 kB 46.7 MB/s 
     |████████████████████████████████| 139 kB 60.8 MB/s 
     |████████████████████████████████| 1.3 MB 54.3 MB/s 
     |████████████████████████████████| 

In [ ]:
from utils import count_words, proportion_unique_words
from utils import strip_punctuation
import pandas as pd
import numpy as np
import pickle5 as pickle
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping
import torch
from sentence_transformers import SentenceTransformer
from transformers import LongformerConfig, LongformerModel, LongformerTokenizer
from transformers import Trainer, TrainingArguments

# Helper Function Definitions

In [ ]:
# Prevent (excessive) scientific notation from being
# printed to console
np.set_printoptions(suppress=True,
   formatter={'float_kind':'{:.5f}'.format})

In [ ]:
def get_sentence_transformer_embedding(lyrics:list):
  encoded_inputs = []
  for lyric in lyrics:
    # Sentence transformers are limited.  From this documentation:
    # https://www.sbert.net/examples/applications/computing-embeddings/README.html
    # `` By default, the provided methods use a limit fo 128 word pieces, 
    # longer inputs will be truncated. [...] Note: You cannot increase the 
    # length higher than what is maximally supported by the respective 
    # transformer model. Also note that if a model was trained on short texts, 
    # the representations for long texts might not be that good.``
    # With this in mind, the max sequence length will be set to 512, 
    # as that is the most that BERT and other transformers have done:
    sent_transformer.max_seq_length = 512
    encoded_lyric = sent_transformer.encode(lyric)
    encoded_inputs.append(encoded_lyric)
  return np.array(encoded_inputs)

In [ ]:
def get_longformer_embedding(lyrics:list):
  encoded_inputs = []
  for lyric in lyrics:
    encoded_input = tokenizer(lyric, return_tensors = "pt", max_length = 1024,
                              truncation = True, padding = "max_length")
    output = longformer(**encoded_input, output_hidden_states = True)
    # Now, the following looks awful (and it is) but essentially this is 
    # what is happening:
    # Take the last layer's output of the longformer [0]
    # Take the [:, 0, :]th slice to get the ``sentence embedding``
    # which ``represents`` the CLS token 
    encoded_lyric = output[0][:, 0, :].detach().numpy()
    # Note that the embeddings might not be meaningful (ie cosine
    # similarity between sentences.) This is limitation is noted in the original 
    # BERT paper and is also a limitation for the longformer.
    encoded_inputs.append(encoded_lyric)
  return np.array(encoded_inputs)

In [ ]:
def get_response_vectors(resp:pd.DataFrame):
  response_vectors = []
  for item in range(len(resp)):
    response_vectors.append( np.array( resp.iloc[item] ) )
  return np.array(response_vectors)

In [ ]:
# Pickle.
def save_vects(vect:np.array, name:str) -> None:
  filename = name + ".pkl"
  if not os.path.exists(filename):
    with open(filename, "wb") as out:
      pickle.dump(vect, out, pickle.HIGHEST_PROTOCOL)
  else:
    print(f"File {filename} already exists in the directory!"
    f" Delete {filename} before attempting to save again.")

In [ ]:
# Load pickle
def load_vects(name:str):
  filename = name + ".pkl"
  return pickle.load(open(filename, "rb"))

In [ ]:
# For implementing checkpointing
def set_up_checkpoint(checkpoint_name):
  return ModelCheckpoint(checkpoint_name, 
                         monitor = "val_acc",
                         mode = 'max',
                         save_best_only = True,
                         save_weights_only = True, 
                         verbose = 1)

# Load in the Data

In [ ]:
x_train = pd.read_csv("data/05_x_train.csv")
x_valid = pd.read_csv("data/05_x_valid.csv")
x_test = pd.read_csv("data/05_x_test.csv")

y_train = pd.read_csv("data/05_y_train.csv")
y_valid = pd.read_csv("data/05_y_valid.csv")
y_test = pd.read_csv("data/05_y_test.csv")

In [ ]:
x_train["prop_unique"] = x_train["lyrics"].apply(proportion_unique_words)
x_train["lyrics"] = strip_punctuation(x_train["lyrics"], for_transformers = True)
x_train["lyrics"] = x_train["lyrics"].str.lower()

x_valid["prop_unique"] = x_valid["lyrics"].apply(proportion_unique_words)
x_valid["lyrics"] = strip_punctuation(x_valid["lyrics"], for_transformers = True)
x_valid["lyrics"] = x_valid["lyrics"].str.lower()

In [ ]:
# Sentence Transformer
# all-mpnet-base-v2 is the best overall performing sent-transformer
# model according to the documentation:
# sbert.net/docs/pretrained_models.html#sentence-embedding-models/
sent_transformer = SentenceTransformer("all-mpnet-base-v2")

Downloading:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/571 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/363 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/13.1k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/349 [00:00<?, ?B/s]

In [ ]:
# Longformer
# https://huggingface.co/docs/transformers/model_doc/longformer
longformer = LongformerModel.from_pretrained("allenai/longformer-base-4096")
tokenizer = LongformerTokenizer.from_pretrained("allenai/longformer-base-4096", 
                                                max_length = 1024)

Some weights of the model checkpoint at allenai/longformer-base-4096 were not used when initializing LongformerModel: ['lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.bias', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.decoder.weight']
- This IS expected if you are initializing LongformerModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing LongformerModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


# Get the Embeddings

In [ ]:
# LF takes forever (~18min for 610 lyrics). This is to be expected.
# TODO: change the loop above to take care of the double nesting of the 
# LF emb output.

x_train_st_emb = get_sentence_transformer_embedding(list(x_train["lyrics"]))
x_train_lf_emb = get_longformer_embedding(list(x_train["lyrics"]))

x_valid_st_emb = get_sentence_transformer_embedding(list(x_valid["lyrics"]))
x_valid_lf_emb = get_longformer_embedding(list(x_valid["lyrics"]))

In [ ]:
y_train_np = get_response_vectors(y_train)
y_valid_np = get_response_vectors(y_valid)

In [ ]:
save_vects(x_train_st_emb, "x_train_st_emb")
save_vects(x_train_lf_emb, "x_train_lf_emb")
save_vects(x_valid_st_emb, "x_valid_st_emb")
save_vects(x_valid_lf_emb, "x_valid_lf_emb")
save_vects(y_train_np, "y_train_np")
save_vects(y_valid_np, "y_valid_np")

File x_train_st_emb.pkl already exists in the directory. Delete x_train_st_emb.pkl before attempting to save again.
File x_train_lf_emb.pkl already exists in the directory. Delete x_train_lf_emb.pkl before attempting to save again.
File x_valid_st_emb.pkl already exists in the directory. Delete x_valid_st_emb.pkl before attempting to save again.
File x_valid_lf_emb.pkl already exists in the directory. Delete x_valid_lf_emb.pkl before attempting to save again.
File y_train_np.pkl already exists in the directory. Delete y_train_np.pkl before attempting to save again.
File y_valid_np.pkl already exists in the directory. Delete y_valid_np.pkl before attempting to save again.


# Make the model

In [ ]:
# Load in the embeddings if they aren't already in the environment.
x_train_st_emb = load_vects("x_train_st_emb")
x_train_lf_emb = load_vects("x_train_lf_emb")
x_valid_st_emb = load_vects("x_valid_st_emb")
x_valid_lf_emb = load_vects("x_valid_lf_emb")
y_train_np = load_vects("y_train_np")
y_valid_np = load_vects("y_valid_np")

In [ ]:
ST_EMB_SIZE = len(x_train_st_emb[0])
LF_EMB_SIZE = len(x_train_lf_emb[0])

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.layers import Dense, BatchNormalization
from tensorflow.keras.initializers import Constant
from keras import layers, Input, Model, Sequential

In [ ]:
N_EPOCHS = 64
N_DIM = len(x_train_st_emb[0])
OBJ_FUNC = "mse"
OPTIMIZER = "adam"
METRICS = ["mae"]
BATCH_SIZE = 1
HIDDEN_ACTIVATION = 'relu'
EARLY_STOP = EarlyStopping(monitor = f"val_{METRICS[0]}", patience = 4, verbose = 1)

In [ ]:
# For implementing checkpointing
def set_up_checkpoint(checkpoint_name):
  return ModelCheckpoint(checkpoint_name, 
                         monitor = f"val_{METRICS[0]}",
                         save_best_only = True,
                         save_weights_only = True, 
                         verbose = 1)

In [ ]:
CKPT = set_up_checkpoint("first_run_dense_dense_2022_08_19")

In [ ]:
X = tf.convert_to_tensor(x_train_st_emb)
Y = tf.convert_to_tensor(y_train_np)

In [ ]:
int_seq_input = Input(shape = (None, BATCH_SIZE, N_DIM), dtype = "float32")
layer1 = Dense(20, activation = 'relu')(int_seq_input)
layer2 = Dense(10, activation = 'relu')(layer1)
out = Dense(7)(layer2)
model = Model(int_seq_input, out)
model.compile(loss = OBJ_FUNC, optimizer = OPTIMIZER, metrics = METRICS)
model.layers[0].trainable = True
model.summary()

Model: "model_52"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_59 (InputLayer)       [(None, None, 1, 768)]    0         
                                                                 
 dense_163 (Dense)           (None, None, 1, 20)       15380     
                                                                 
 dense_164 (Dense)           (None, None, 1, 10)       210       
                                                                 
 dense_165 (Dense)           (None, None, 1, 7)        77        
                                                                 
Total params: 15,667
Trainable params: 15,667
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.fit(X, Y, batch_size = BATCH_SIZE, 
          epochs = N_EPOCHS, validation_data = (x_valid_st_emb, y_valid_np), 
          callbacks = [EARLY_STOP, CKPT])

Epoch 1/64


591/610 [============================>.] - ETA: 0s - loss: 5.9336 - mae: 1.0879


Epoch 1: val_mae improved from inf to 0.71229, saving model to first_run_dense_dense_2022_08_19
610/610 [==============================] - 2s 3ms/step - loss: 5.8247 - mae: 1.0754 - val_loss: 2.1645 - val_mae: 0.7123
Epoch 2/64
598/610 [============================>.] - ETA: 0s - loss: 2.2958 - mae: 0.7169
Epoch 2: val_mae did not improve from 0.71229
610/610 [==============================] - 2s 3ms/step - loss: 2.2865 - mae: 0.7164 - val_loss: 2.1124 - val_mae: 0.7177
Epoch 3/64
602/610 [============================>.] - ETA: 0s - loss: 2.1794 - mae: 0.7082
Epoch 3: val_mae did not improve from 0.71229
610/610 [==============================] - 2s 3ms/step - loss: 2.1827 - mae: 0.7088 - val_loss: 2.0875 - val_mae: 0.7313
Epoch 4/64
609/610 [============================>.] - ETA: 0s - loss: 2.1169 - mae: 0.7015
Epoch 4: val_mae did not improve from 0.71229
610/610 [==============================] - 2s 3ms/step - loss: 2.1161 - mae: 0.7014 - val_loss: 2.0579 - val_mae: 0.7124
Epoch 5/

In [ ]:
y_test_np = np.array(y_test)
x_test_st_emb = get_sentence_transformer_embedding(x_test)

In [ ]:
y_test_hat = model.predict(x_test_st_emb, verbose = 0)

In [ ]:
y_test_hat[1]

array([0.26675, 0.56818, 0.20460, -5.52965, -0.97212, -1.31918, 0.23921],
      dtype=float32)

In [ ]:
y_test_np[1]

array([0.25800, 0.85300, 0.69800, -4.36607, -1.64506, -3.10776, 0.53100])